# Isogenies

**Note: due to the context of this being explained during Sage days, I assume you are all running the most up to date version of SageMath at the time of writing (version 10.2)**

This notebook assumes you have already gone through, or are familiar with using elliptic curves in Sage. 

## Isomorphisms

## Isogenies from Points

## Isogenies from Kernel Polynomials

## Isogenies Between Curves

## Smooth Degree Isogenies

## Large Degree Isogenies